In [163]:
from typing import Dict, List, Callable, Literal
import enum
import heapq
import numpy as np

### Task 1

In [164]:
type Components = Dict[str, Literal[0, 1]]

class Environment:
    def __init__(self):
        self.__components: Components = \
            dict([(chr(ord('A') + i), np.random.randint(0, 2)) for i in range(9)])
        
    def display_state(self):
        print(self.__components)
    
    def get_percept(self) -> Components:
        return self.__components
    
    def patch(self, component: str):
        self.__components.update({component: 0})

class Agent:
    def __init__(self):
        self.__patches: List[str] = []
    
    def scan(self, components: Components):
        for component in components:
            if components[component] == 0:
                print(f'<success>: {component} is secure.')
                continue
            print(f'<warning>: {component} is not secure!')
            self.__patches.append(component)

    def act(self, patch: Callable[[str], None]):
        for component in self.__patches:
            patch(component)
        self.__patches.clear()

def run_agent():
    env = Environment()
    agent = Agent()

    env.display_state()
    agent.scan(env.get_percept())
    agent.act(env.patch)
    env.display_state()

run_agent()

{'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 1, 'F': 1, 'G': 0, 'H': 1, 'I': 1}
<success>: A is secure.
<success>: B is secure.
<warning>: C is not secure!
<warning>: D is not secure!
<warning>: E is not secure!
<warning>: F is not secure!
<success>: G is secure.
<warning>: H is not secure!
<warning>: I is not secure!
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0}


### Task 2

In [161]:
class States(enum.Enum):
    UNDERLOADED = 'UNDERLOADED'
    BALANCED = 'BALANCED'
    OVERLOADED = 'OVERLOADED'

BALANCED_COUNT = 10

class Server:
    def __init__(self, initial_tasks: List[int] = []):
        self.__tasks: List[int] = []
        self.__state = States.UNDERLOADED
        for task in initial_tasks:
            self.add(task)
    
    def add(self, task: int):
        self.__tasks.append(task)
        self.__update_state()

    def remove(self) -> int:
        task: int = self.__tasks.pop()
        self.__update_state()
        return task

    def get_state(self) -> States:
        return self.__state

    def __str__(self):
        return f'status: {self.__state.value:<11} | tasks[{len(self.__tasks)}]: {', '.join(map(str, self.__tasks))}'
    
    def __gt__(self, obj):
        return len(self.__tasks) < len(obj.__tasks)
    
    def __len__(self):
        return len(self.__tasks)
    
    def __update_state(self):
        n = len(self.__tasks)
        if n > BALANCED_COUNT:
            self.__state = States.OVERLOADED
        elif n == BALANCED_COUNT:
            self.__state = States.BALANCED
        else:
            self.__state = States.UNDERLOADED

class Environment:
    def __init__(self):
        self.__servers: List[Server] = [
            Server(np.random.randint(0, 20, np.random.randint(15))) for _ in range(5)
        ]

    def display_state(self):
        total = 0
        for i, server in enumerate(self.__servers):
            print(f'{i}:', server)
            total += len(server)
        print('Total tasks:', total)
    
    def update_servers(self, servers: List[Server]):
        self.__servers = servers

    def get_precept(self) -> List[Server]:
        return self.__servers

class Agent:
    class ServerWrapper:
        def __init__(self, server: Server):
            self.server = server
        def __gt__(self, obj):
            return len(self.server) > len(obj.server)

    def __init__(self):
        self.__balanced: List[Server] = []
        self.__overloaded_pq: List[Server] = []
        self.__underloaded_pq: List[Agent.ServerWrapper] = []

    def scan(self, servers: List[Server]):
        for server in servers:
            if server.get_state() == States.OVERLOADED:
                heapq.heappush(self.__overloaded_pq, server)
            elif server.get_state() == States.UNDERLOADED:
                heapq.heappush(self.__underloaded_pq, Agent.ServerWrapper(server))
            else:
                self.__balanced.append(server)
    
    def act(self):
        while self.__overloaded_pq and self.__underloaded_pq:
            overloaded: Server = heapq.heappop(self.__overloaded_pq)
            underloaded: Agent.ServerWrapper = heapq.heappop(self.__underloaded_pq)

            while len(overloaded) > BALANCED_COUNT and len(underloaded.server) < BALANCED_COUNT:
                underloaded.server.add(overloaded.remove())

            if overloaded.get_state() != States.BALANCED:
                heapq.heappush(self.__overloaded_pq, overloaded)
            else:
                self.__balanced.append(overloaded)

            if underloaded.server.get_state() != States.BALANCED:
                heapq.heappush(self.__underloaded_pq, underloaded)
            else:
                self.__balanced.append(underloaded.server)
        
        return self.__balanced + self.__overloaded_pq + list(map(lambda s: s.server, self.__underloaded_pq))

def run_agent():
    env = Environment()
    agent = Agent()
    
    env.display_state()
    agent.scan(env.get_precept())
    env.update_servers(agent.act())
    env.display_state()

run_agent()

0: status: OVERLOADED  | tasks[12]: 9, 10, 11, 9, 8, 2, 3, 10, 18, 2, 15, 7
1: status: UNDERLOADED | tasks[4]: 1, 18, 14, 15
2: status: OVERLOADED  | tasks[12]: 14, 2, 7, 6, 18, 7, 15, 19, 19, 5, 11, 2
3: status: OVERLOADED  | tasks[11]: 1, 9, 13, 12, 0, 11, 9, 15, 5, 7, 6
4: status: UNDERLOADED | tasks[6]: 11, 14, 13, 9, 14, 9
Total tasks: 45
0: status: BALANCED    | tasks[10]: 9, 10, 11, 9, 8, 2, 3, 10, 18, 2
1: status: BALANCED    | tasks[10]: 14, 2, 7, 6, 18, 7, 15, 19, 19, 5
2: status: BALANCED    | tasks[10]: 1, 9, 13, 12, 0, 11, 9, 15, 5, 7
3: status: UNDERLOADED | tasks[7]: 1, 18, 14, 15, 7, 15, 6
4: status: UNDERLOADED | tasks[8]: 11, 14, 13, 9, 14, 9, 2, 11
Total tasks: 45


### Task 3